In [3]:
import pandas as pd
import numpy as np

bitcoin_train = pd.read_csv('/content/drive/MyDrive/Bitcoin_data/bitcoin_train_raw.csv')  
bitcoin_val = pd.read_csv('/content/drive/MyDrive/Bitcoin_data/bitcoin_val_raw.csv') 
bitcoin_test = pd.read_csv('/content/drive/MyDrive/Bitcoin_data/bitcoin_test_raw.csv')

bitcoin_processed = pd.concat([bitcoin_train, bitcoin_val , bitcoin_test], axis = 0 )

bitcoin_high_val = bitcoin_processed[['date' , 'high']]

#bitcoin_high_val = pd.concat([bitcoin_extra , bitcoin_high_val ] , axis= 0).reset_index(drop = True)

bitcoin_processed.head()

,Unnamed: 0,date,high,1m return,1m class,5m return,5m class,15m return,15m class,60m return,60m class,1week return
0,0,2019-03-04 00:00:00,3789.70,0.000000,1,0.000881,0,0.003470,0,0.003879,0,0.029081
1,1,2019-03-04 00:01:00,3789.70,0.000251,0,0.002074,0,0.004539,0,0.004143,0,0.029055
2,2,2019-03-04 00:02:00,3790.65,0.000586,0,0.001195,0,0.003559,0,0.003464,0,0.028797
3,3,2019-03-04 00:03:00,3792.87,0.000045,0,0.000609,0,0.003043,0,0.003056,0,0.028329
4,4,2019-03-04 00:04:00,3793.04,0.000000,1,0.000564,0,0.001598,0,0.002681,0,0.028283


In [4]:
tweets_features_all = pd.read_csv('/content/drive/MyDrive/Bitcoin_data/The features of twitter.csv')
tweets_features_all.drop(tweets_features_all.columns[0],axis=1, inplace=True)

tweets_features_all.head()

,date,num_tweets,sum_score,Weighted_Sentiment_Score
0,2019-03-04 00:00:00,0,0.0,0.00
1,2019-03-04 00:01:00,0,0.0,0.00
2,2019-03-04 00:02:00,0,0.0,0.00
3,2019-03-04 00:03:00+00:00,1,0.2,0.26
4,2019-03-04 00:04:00+00:00,0,0.0,0.00


In [5]:
from pandas.core.arrays import interval
import numpy as np

intervals = np.array([[0, 1], [1, 2], [2, 3], [3, 4], [4, 5],  [5, 10], [10, 20], [20, 40], [40, 60], [60, 80], [80, 100], [100, 120]])
multi_period_returns = []

bitcoin_high_vals = np.array(bitcoin_high_val['high'])
num_tweets_values = np.array(tweets_features_all['num_tweets'])
sum_scores_values = np.array(tweets_features_all['sum_score'])
weight_senti_values = np.array(tweets_features_all['Weighted_Sentiment_Score'])

num_tweets_features = []

sum_scores_features = []

weight_senti_features = []

for i in range(120 , len(bitcoin_high_val)):

    interval_vals = bitcoin_high_vals[i - intervals[:,0]] / bitcoin_high_vals[i - intervals[:,1]]
    temp = [(val - 1) for val in interval_vals]
    multi_period_returns.append(temp)

    
    temp_intervals = i - intervals

    temp = [sum(sum_scores_values[start : end]) for end, start in temp_intervals]
    sum_scores_features.append(temp)

    temp = [sum(num_tweets_values[start : end]) for end, start in temp_intervals]
    num_tweets_features.append(temp)

    temp = [sum(weight_senti_values[start : end]) for end, start in temp_intervals]
    weight_senti_features.append(temp)


multi_period_returns = np.array(multi_period_returns)
num_tweets_features = np.array(num_tweets_features)
sum_scores_features = np.array(sum_scores_features)
weight_senti_features = np.array(weight_senti_features)



In [6]:
dataset = pd.DataFrame(np.concatenate([multi_period_returns, num_tweets_features, sum_scores_features, weight_senti_features ] , axis = 1))


dataset.columns = (["btc_return_%d-%dmin" % (end , start) for end , start in intervals ] +  
                   
["n_tweets_%d-%dmin"  % (end , start) for end , start in intervals] +

["sentiment_%d-%dmin"  % (end , start) for end , start in intervals]+

["sentiment_wt_%d-%dmin"  % (end , start) for end , start in intervals])


dataset.insert(0, 'time' , bitcoin_high_val[120:].reset_index()['date'])
dataset.insert(1 , '1week return' , bitcoin_processed[120:].reset_index()['1week return'])
dataset.describe()

,1week return,btc_return_0-1min,btc_return_1-2min,btc_return_2-3min,btc_return_3-4min,btc_return_4-5min,btc_return_5-10min,btc_return_10-20min,btc_return_20-40min,btc_return_40-60min,...,sentiment_wt_2-3min,sentiment_wt_3-4min,sentiment_wt_4-5min,sentiment_wt_5-10min,sentiment_wt_10-20min,sentiment_wt_20-40min,sentiment_wt_40-60min,sentiment_wt_60-80min,sentiment_wt_80-100min,sentiment_wt_100-120min
count,405960.000000,405960.000000,405960.000000,405960.000000,405960.000000,405960.000000,405960.000000,405960.000000,405960.000000,405960.000000,...,405960.000000,405960.000000,405960.000000,405960.000000,405960.000000,405960.000000,405960.000000,405960.000000,405960.000000,405960.000000
mean,0.020910,0.000002,0.000002,0.000002,0.000002,0.000002,0.000011,0.000021,0.000043,0.000043,...,-0.001258,-0.001258,-0.001258,-0.006290,-0.012573,-0.025087,-0.025086,-0.025070,-0.025087,-0.025070
std,0.108126,0.000987,0.000987,0.000987,0.000987,0.000987,0.002370,0.003351,0.004701,0.004701,...,0.204704,0.204704,0.204704,0.466853,0.670335,0.966215,0.966215,0.966216,0.966223,0.966216
min,-0.304505,-0.076791,-0.076791,-0.076791,-0.076791,-0.076791,-0.094700,-0.159965,-0.180152,-0.180152,...,-4.000000,-4.000000,-4.000000,-4.510000,-5.740000,-7.480000,-7.480000,-7.480000,-7.480000,-7.480000
25%,-0.045595,-0.000361,-0.000361,-0.000361,-0.000361,-0.000361,-0.000856,-0.001169,-0.001576,-0.001576,...,0.000000,0.000000,0.000000,-0.040000,-0.220000,-0.470000,-0.470000,-0.470000,-0.470000,-0.470000
50%,0.000931,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000010,-0.000015,-0.000010,-0.000010,...,0.000000,0.000000,0.000000,0.000000,0.000000,-0.010000,-0.010000,-0.010000,-0.010000,-0.010000
75%,0.071128,0.000325,0.000325,0.000325,0.000325,0.000325,0.000810,0.001146,0.001583,0.001583,...,0.000000,0.000000,0.000000,0.000000,0.100000,0.330000,0.330000,0.330000,0.330000,0.330000
max,0.521888,0.087718,0.087718,0.087718,0.087718,0.087718,0.127636,0.130052,0.158624,0.158624,...,6.770000,6.770000,6.770000,8.730000,8.730000,10.720000,10.720000,10.720000,10.720000,10.720000


In [7]:
import pickle

with open('/content/drive/MyDrive/Bitcoin_data/dataset.pickle', 'wb') as file:

    pickle.dump(dataset, file)


labels = bitcoin_processed[120:].reset_index()[['date' , '5m class' , '1m class' , '15m class' , '60m class']]

with open('/content/drive/MyDrive/Bitcoin_data/labels.pickle', 'wb') as file:

    pickle.dump(labels, file)


In [8]:
import pickle as pkl

with open('/content/drive/MyDrive/Bitcoin_data/dataset.pickle', 'rb') as f:
    dataset = pkl.load(f)

with open('/content/drive/MyDrive/Bitcoin_data/labels.pickle', 'rb') as f:
    labels = pkl.load(f)


In [10]:
dataset.shape , labels.shape 

((405960, 50), (405960, 5))